In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directory paths
train_data_dir = '/content/drive/MyDrive/data/train'
validation_data_dir = '/content/drive/MyDrive/data/val'

# Image dimensions
img_width, img_height = 160, 160

# Batch size
batch_size = 32

# Data augmentation parameters
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=[0.8, 1],
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Generating image data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    classes=['Brown rust', 'Healthy', 'Yellow rust'],  # Adjusted to 3 classes
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    classes=['Brown rust', 'Healthy', 'Yellow rust'],  # Adjusted to 3 classes
    class_mode='categorical'
)

# Build the CNN model with VGG16 base
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

model = models.Sequential()
model.add(vgg_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))  # Adjusted output classes to 3

# Freeze the VGG16 base layers
vgg_base.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training parameters
epochs = 3

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Save the trained model
model.save('/content/CNNModel_saved.h5')

Found 1116 images belonging to 3 classes.
Found 279 images belonging to 3 classes.
Epoch 1/3
34/34 [==============================] - 256s 8s/step - loss: 0.0500 - accuracy: 0.9742 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/3
34/34 [==============================] - 23s 690ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/3
34/34 [==============================] - 22s 661ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directory path for training and validation data
train_data_dir = '/content/drive/MyDrive/data/train'
validation_data_dir = '/content/drive/MyDrive/data/val'

# Image dimensions
img_width, img_height = 256, 256

# Batch size
batch_size = 32

# Data augmentation parameters for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=[0.8, 1],
    horizontal_flip=True,  # Enable horizontal flipping
    fill_mode='nearest'     # Fill mode for filling in newly created pixels
)

# Data augmentation parameters for validation data (only rescaling)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Generating image data generators for training and validation data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    classes=['Brown_rust', 'Healthy', 'Yellow_rust'],  # Adjusted to 3 classes
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    classes=['Brown_rust', 'Healthy', 'Yellow_rust'],  # Adjusted to 3 classes
    class_mode='categorical'
)

# Define the CNN architecture
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))  # Additional Dense layer
model.add(layers.Dropout(0.5))  # Dropout layer for regularization
model.add(layers.Dense(len(train_generator.class_indices), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Training the model
epochs = 10  # You need to define the number of epochs
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Saving the model
model.save('/content/drive/MyDrive/CNNModel_saved.h5')


Found 2942 images belonging to 3 classes.
Found 737 images belonging to 3 classes.
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 127, 127, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 60, 60, 128)       73856     
                                     

In [14]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load your trained model
model_path = '/content/drive/MyDrive/CNNModel_saved.h5'  # Update with the path to your saved model
model = load_model(model_path)

# Path to the single image you want to predict
image_path = '/content/drive/MyDrive/data/train/Yellow_rust/Yellow_rust025.jpg'

# Load and preprocess the image
img = image.load_img(image_path, target_size=(img_width, img_height))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Rescale to [0,1]

# Predict the disease class
prediction = model.predict(img_array)
predicted_class_index = np.argmax(prediction)
class_labels = ['Brown_rust', 'Healthy', 'Yellow_rust']
predicted_class = class_labels[predicted_class_index]

print("Predicted class:", predicted_class)


1/1 [==============================] - 0s 80ms/step
Predicted class: Yellow_rust
